In [2]:
import pandas as pd
from datetime import timedelta
import numpy as np

# -----------------
# Define Resource Catalog
resources_catalog = {
    "Smoke Jumpers": {"deployment": 30*60, "cost": 5000, "units": 5},
    "Fire Engines": {"deployment": 60*60, "cost": 2000, "units": 10},
    "Helicopters": {"deployment": 45*60, "cost": 8000, "units": 3},
    "Tanker Planes": {"deployment": 120*60, "cost": 15000, "units": 2},
    "Ground Crews": {"deployment": 90*60, "cost": 3000, "units": 8},
}

damage_costs = {"low": 50000, "medium": 100000, "high": 200000}
severity_rank = {"high": 3, "medium": 2, "low": 1}

# -----------------
# Simulation Function
def simulate_deployment(df, resources_catalog, order_fn):
    # Create a copy of catalog availability: each resource gets a list of available times (initially 0)
    resource_availability = {
        rname: [0] * prop["units"] for rname, prop in resources_catalog.items()
    }
    # Counters
    addressed = {"low": 0, "medium": 0, "high": 0}
    missed = {"low": 0, "medium": 0, "high": 0}
    operational_cost = 0
    damage_cost = 0
    
    # Convert timestamps to Unix seconds for easy comparison
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['fire_start_time'] = pd.to_datetime(df['fire_start_time'])
    df['report_sec'] = df['timestamp'].astype('int64') // 10**9

    # Sort events by report time, then severity (high first)
    df['severity_rank'] = df['severity'].map(severity_rank)
    df = df.sort_values(by=['report_sec', 'severity_rank'], ascending=[True, False])
    
    # Get resource ordering based on provided order function
    resource_order = order_fn(resources_catalog)

    for _, row in df.iterrows():
        report_time = row['report_sec']
        sev = row['severity']
        assigned = False
        
        # Try assigning a resource using the chosen ordering strategy.
        for rname, props in resource_order:
            avail_list = resource_availability[rname]
            for i, avail_time in enumerate(avail_list):
                if avail_time <= report_time:
                    # Assign resource: update availability time by adding deployment duration.
                    resource_availability[rname][i] = report_time + props["deployment"]
                    operational_cost += props["cost"]
                    addressed[sev] += 1
                    assigned = True
                    break
            if assigned:
                break
        
        if not assigned:
            missed[sev] += 1
            damage_cost += damage_costs[sev]

    total_cost = operational_cost + damage_cost
    result = {
        "addressed": addressed,
        "missed": missed,
        "operational_cost": operational_cost,
        "damage_cost": damage_cost,
        "total_cost": total_cost
    }
    return result

# -----------------
# Strategy 1: Order by Operational Cost (cheapest first)
def order_by_cost(resources):
    return sorted(resources.items(), key=lambda x: x[1]['cost'])

# Strategy 2: Order by Deployment Time (fastest first)
def order_by_deployment(resources):
    return sorted(resources.items(), key=lambda x: x[1]['deployment'])

# -----------------
# Load Historical Data (Use cleaned data file)
hist_path = "/Users/duchuyta/Downloads/SAP-CONUHACKS-IX/Data/Preprocessed_1/historical_wildfiredata_cleaned.csv"
historical_df = pd.read_csv(hist_path)

# Run simulations on historical data for both strategies
result_cost = simulate_deployment(historical_df.copy(), resources_catalog, order_by_cost)
result_deployment = simulate_deployment(historical_df.copy(), resources_catalog, order_by_deployment)

print("Historical Data Simulation Results:")
print("Strategy Order by Cost:", result_cost)
print("Strategy Order by Deployment Time:", result_deployment)

# Choose best strategy (lowest total cost)
if result_cost["total_cost"] <= result_deployment["total_cost"]:
    best_strategy = order_by_cost
    best_name = "Order by Cost"
else:
    best_strategy = order_by_deployment
    best_name = "Order by Deployment Time"

print(f"Chosen Best Strategy: {best_name}")

# -----------------
# Now, Test Best Strategy on Current Data
current_path = "/Users/duchuyta/Downloads/SAP-CONUHACKS-IX/Data/Preprocessed_1/current_wildfiredata_cleaned.csv"
current_df = pd.read_csv(current_path)
# Ensure timestamp columns are parsed in the simulation function

current_result = simulate_deployment(current_df.copy(), resources_catalog, best_strategy)

print("\nCurrent Data Simulation Results:")
print("Number of fires addressed:", sum(current_result["addressed"].values()))
print("Number of fires missed:", sum(current_result["missed"].values()))
print("Operational costs: $", current_result["operational_cost"])
print("Estimated damage costs: $", current_result["damage_cost"])
print("Fire severity report (addressed):", current_result["addressed"])
print("Fire severity report (missed):", current_result["missed"])

Historical Data Simulation Results:
Strategy Order by Cost: {'addressed': {'low': 65, 'medium': 38, 'high': 30}, 'missed': {'low': 0, 'medium': 0, 'high': 0}, 'operational_cost': 266000, 'damage_cost': 0, 'total_cost': 266000}
Strategy Order by Deployment Time: {'addressed': {'low': 65, 'medium': 38, 'high': 30}, 'missed': {'low': 0, 'medium': 0, 'high': 0}, 'operational_cost': 665000, 'damage_cost': 0, 'total_cost': 665000}
Chosen Best Strategy: Order by Cost

Current Data Simulation Results:
Number of fires addressed: 32
Number of fires missed: 0
Operational costs: $ 64000
Estimated damage costs: $ 0
Fire severity report (addressed): {'low': 14, 'medium': 11, 'high': 7}
Fire severity report (missed): {'low': 0, 'medium': 0, 'high': 0}
